Baseball Prediction: 1 - Data Wrangling

In this notebook, we will wrangle data downloaded from www.retrosheet.org into a dataframe suitable for model building. Specifically, for each game, we will calculate some team statistics over their past 162 games.

At the end, we save our dataframe to a file. This file will be the starting point for the next notebook, in which we build our first model.

To use this notebook, you must first download the game logs here: https://www.retrosheet.org/gamelogs/index.html Towards the bottom of the page there are links for ZIP files containing multiple seasons. Download the zip files containing the 'gamelog' data, decompress it, and then move all of the single season files to a single directory. You will then need the path to that directory for the variable fname below.

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)

In [9]:
fname = '/Users/gilliancurtis/Downloads/alldata/gamelogs/'+'gl2022.txt'
df = pd.read_csv(fname, header=None)

In [10]:
df.head(25)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160
0,20220407,0,Thu,SDN,NL,1,ARI,NL,1,2,4,51,N,NaN,NaN,NaN,PHO01,35508.0,198,001100000,000000004,29,4,1,0,0,2,0,0,0,7,0,5,1,0,2,0,7,5,4,4,2,0,24,8,0,0,1,0,26,3,0,0,1,3,0,0,1,7,0,6,0,0,1,0,6,6,2,2,0,0,27,9,0,0,2,0,vanol901,Larry Vanover,belld901,Dan Bellino,barbs901,Sean Barber,valej901,Junior Valentine,NaN,(none),NaN,(none),melvb001,Bob Melvin,lovut001,Tony Lovullo,mantj002,Joe Mantiply,suarr002,Robert Suarez,NaN,(none),beers001,Seth Beer,darvy001,Yu Darvish,bumgm001,Madison Bumgarner,nolaa002,Austin Nola,2,machm001,Manny Machado,5,cronj001,Jake Cronenworth,4,voitl001,Luke Voit,10,myerw001,Wil Myers,9,hosme001,Eric Hosmer,3,profj001,Jurickson Profar,7,kim-h002,Ha-Seong Kim,6,grist001,Trent Grisham,8,varsd001,Daulton Varsho,8,martk001,Ketel Marte,4,perad001,David Peralta,7,walkc002,Christian Walker,3,smitp002,Pavin Smith,9,kellc002,Carson Kelly,2,beers001,Seth Beer,10,ellid002,Drew Ellis,5,perdg001,Gerardo Perdomo,6,NaN,Y
1,20220407,0,Thu,CIN,NL,1,ATL,NL,1,6,3,54,N,NaN,NaN,NaN,ATL03,40545.0,181,012003000,001000020,35,10,0,0,1,6,0,1,1,1,0,13,0,0,1,0,5,5,2,2,0,0,27,7,1,0,0,0,31,4,0,0,1,2,0,0,0,5,0,13,0,0,0,0,6,4,6,6,0,0,27,7,0,0,1,0,laynj901,Jerry Layne,wendh902,Hunter Wendelstedt,whitc901,Chad Whitson,hamaa901,Adam Hamari,NaN,(none),NaN,(none),belld002,David Bell,snitb801,Brian Snitker,mahlt001,Tyler Mahle,friem001,Max Fried,santt001,Tony Santillan,farmk001,Kyle Farmer,mahlt001,Tyler Mahle,friem001,Max Fried,indij001,Jonathan India,4,aquia001,Aristides Aquino,9,phamt001,Tommy Pham,7,vottj001,Joey Votto,3,stept001,Tyler Stephenson,2,senzn001,Nick Senzel,8,mousm001,Mike Moustakas,10,farmk001,Kyle Farmer,6,drurb001,Brandon Drury,5,rosae001,Eddie Rosario,9,olsom001,Matt Olson,3,rilea001,Austin Riley,5,ozunm001,Marcell Ozuna,7,albio001,Ozzie Albies,4,duvaa001,Adam Duvall,8,darnt001,Travis d'Arnaud,2,dicka001,Alex Dickerson,10,swand001,Dansby Swanson,6,NaN,Y
2,20220407,0,Thu,MIL,NL,1,CHN,NL,1,4,5,51,D,NaN,NaN,NaN,CHI11,35112.0,198,000100210,00003020x,33,10,4,0,0,4,0,2,1,4,0,9,1,0,2,0,9,3,5,5,0,0,24,10,0,0,1,0,29,8,3,0,1,5,0,1,1,4,0,7,0,1,1,0,6,6,4,4,0,0,27,9,0,0,2,0,barrt901,Ted Barrett,barkl901,Lance Barksdale,lentn901,Nic Lentz,cejan901,Nestor Ceja,NaN,(none),NaN,(none),counc001,Craig Counsell,rossd001,David Ross,givem001,Mychal Givens,ashba003,Aaron Ashby,robed002,David Robertson,happi001,Ian Happ,burnc002,Corbin Burnes,hendk001,Kyle Hendricks,wongk001,Kolten Wong,4,adamw002,Willy Adames,6,yelic001,Christian Yelich,7,mccua001,Andrew McCutchen,10,tellr001,Rowdy Tellez,3,renfh001,Hunter Renfroe,9,narvo001,Omar Narvaez,2,cainl001,Lorenzo Cain,8,petej002,Jace Peterson,5,orter001,Rafael Ortega,10,madrn001,Nick Madrigal,4,contw001,Willson Contreras,2,happi001,Ian Happ,7,schwf001,Frank Schwindel,3,suzus001,Seiya Suzuki,9,heywj001,Jason Heyward,8,wisdp001,Patrick Wisdom,5,hoern001,Nico Hoerner,6,NaN,Y
3,20220407,0,Thu,PIT,NL,1,SLN,NL,1,0,9,51,D,NaN,NaN,NaN,STL10,46256.0,188,000000000,13000104x,30,6,0,0,0,0,0,0,0,2,0,9,0,1,2,0,5,6,9,9,0,0,24,7,2,0,1,0,31,8,2,0,3,9,0,2,1,7,0,5,1,0,1,0,8,4,0,0,1,0,27,11,0,0,3,0,reynj901,Jim Reynolds,ticht901,Todd Tichenor,muchm901,Mike Muchlinski,merzd901,Dan Merzel,NaN,(none),NaN,(none),sheld801,Derek Shelton,marmo801,Oliver Marmol,waina001,Adam Wainwright,brubj001,JT Brubaker,NaN,(none),oneit001,Tyler O'Neill,brubj001,JT Brubaker,waina001,Adam Wainwright,voged001,Daniel Vogelbach,10,reynb001,Bryan Reynolds,8,hayek001,Ke'Bryan Hayes,5,tsuty001,Yoshi Tsutsugo,3,newmk001,Kevin Newman,6,tu

In [11]:
colnames = ['date','dblheader_code','day_of_week','team_v','league_v','game_no_v',
           'team_h','league_h','game_no_h', 'runs_v', 'runs_h','outs_total','day_night',
            'completion_info','forfeit_info','protest_info','ballpark_id','attendance','game_minutes',
            'linescore_v','linescore_h',
           'AB_v','H_v','2B_v','3B_v','HR_v','RBI_v','SH_v','SF_v','HBP_v','BB_v','IBB_v','SO_v',
            'SB_v', 'CS_v','GIDP_v','CI_v','LOB_v',
            'P_num_v','ERind_v','ERteam_v','WP_v','balk_v',
            'PO_v','ASST_v','ERR_v','PB_v','DP_v','TP_v',
           'AB_h', 'H_h', '2B_h', '3B_h', 'HR_h', 'RBI_h', 'SH_h', 'SF_h', 'HBP_h', 'BB_h', 'IBB_h','SO_h',
            'SB_h', 'CS_h', 'GIDP_h', 'CI_h', 'LOB_h',
            'P_num_h', 'ERind_h', 'ERteam_h', 'WP_h', 'balk_h',
            'PO_h', 'ASST_h', 'ERR_h', 'PB_h', 'DP_h', 'TP_h',
            'ump_HB_id', 'ump_HB_name','ump_1B_id', 'ump_1B_name','ump_2B_id', 'ump_2B_name',
            'ump_3B_id', 'ump_3B_name','ump_LF_id', 'ump_LF_name','ump_RF_id', 'ump_RF_name',
            'mgr_id_v', 'mgr_name_v', 'mgr_id_h', 'mgr_name_h',
            'pitcher_id_w','pitcher_name_w','pitcher_id_l','pitcher_name_l','pitcher_id_s','pitcher_name_s',
            'GWRBI_id','GWRBI_name','pitcher_start_id_v','pitcher_start_name_v','pitcher_start_id_h','pitcher_start_name_h',
            'batter1_name_v', 'batter1_id_v', 'batter1_pos_v', 'batter2_name_v', 'batter2_id_v', 'batter2_pos_v',
            'batter3_name_v', 'batter3_id_v', 'batter3_pos_v', 'batter4_name_v', 'batter4_id_v', 'batter4_pos_v',
            'batter5_name_v', 'batter5_id_v', 'batter5_pos_v', 'batter6_name_v', 'batter6_id_v', 'batter6_pos_v',
            'batter7_name_v', 'batter7_id_v', 'batter7_pos_v', 'batter8_name_v', 'batter8_id_v', 'batter8_pos_v',
            'batter9_name_v', 'batter9_id_v', 'batter9_pos_v', 'batter1_name_h', 'batter1_id_h', 'batter1_pos_h',
            'batter2_name_h', 'batter2_id_h', 'batter2_pos_h', 'batter3_name_h', 'batter3_id_h', 'batter3_pos_h',
            'batter4_name_h', 'batter4_id_h', 'batter4_pos_h', 'batter5_name_h', 'batter5_id_h', 'batter5_pos_h',
            'batter6_name_h', 'batter6_id_h', 'batter6_pos_h', 'batter7_name_h', 'batter7_id_h', 'batter7_pos_h',
            'batter8_name_h', 'batter8_id_h', 'batter8_pos_h', 'batter9_name_h', 'batter9_id_h', 'batter9_pos_h',           
           'misc_info','acqui_info'
           ]

df.columns = colnames

df.sample(25)

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info
132,20220417,0,Sun,ARI,NL,9,NYN,NL,10,0,5,51,D,NaN,NaN,NaN,NYC20,24515.0,179,000000000,00000320x,32,5,2,0,0,0,0,0,0,4,0,10,0,0,0,0,9,5,5,5,0,0,24,6,1,0,0,0,29,6,1,0,1,4,0,1,0,6,0,2,2,0,0,0,7,5,0,0,0,0,27,5,0,0,0,0,gonzm901,Manny Gonzalez,diazl901,Laz Diaz,onorb901,Brian O'Nora,fairc901,Chad Fairchild,NaN,(none),NaN,(none),lovut001,Tony Lovullo,showb801,Buck Showalter,shrec001,Chasen Shreve,ramin002,Noe Ramirez,NaN,(none),NaN,(none),casth002,Humberto Castellanos,peted003,David Peterson,hummc001,Cooper Hummel,7,martk001,Ketel Marte,4,kellc002,Carson Kelly,10,walkc002,Christian Walker,3,smitp002,Pavin Smith,9,varsd001,Daulton Varsho,2,herny003,Yonny Hernandez,5,mccaj003,Jake McCarthy,8,perdg001,Gerardo Perdomo,6,mcnej002,Jeff McNeil,7,marts002,Starling Marte,9,lindf001,Francisco Lindor,6,alonp001,Pete Alonso,10,escoe001,Eduardo Escobar,5,smitd008,Dominic Smith,3,jankt001,Travis Jankowski,8,mccaj001,James McCann,2,guill001,Luis Guillorme,4,NaN,Y
1540,20220801,0,Mon,BAL,AL,103,TEX,AL,102,7,2,54,N,NaN,NaN,NaN,ARL03,19161.0,175,130200100,000001010,40,12,4,0,1,7,0,0,1,3,0,7,1,0,0,0,10,2,2,2,0,0,27,12,0,0,1,0,33,7,0,0,2,2,0,0,0,0,0,8,0,0,1,0,4,5,6,6,0,0,27,8,1,0,0,0,barrs901,Scott Barry,iassd901,Dan Iassogna,rehaj901,Jeremie Rehak,becka901,Adam Beck,NaN,(none),NaN,(none),hydeb801,Brandon Hyde,woodc001,Chris Woodward,watks003,Spenser Watkins,grayj003,Jon Gray,akink001,Keegan Akin,mullc002,Cedric Mullins,watks003,Spenser Watkins,grayj003,Jon Gray,mullc002,Cedric Mullins,8,rutsa001,Adley Rutschman,2,santa003,Anthony Santander,10,mounr001,Ryan Mountcastle,3,odorr001,Rougned Odor,4,uriar001,Ramon Urias,5,vavrt001,Terrin Vavra,7,mcker001,Ryan McKenna,9,matej003,Jorge Mateo,6,semim001,Marcus Semien,6,seagc001,Corey Seager,10,heimj001,Jonah Heim,2,lowen001,Nate Lowe,3,garca005,Adolis Garcia,9,tavel001,Leody Taveras,8,calhk001,Kole Calhoun,7,durae002,Ezequiel Duran,4,smitj007,Josh Smith,5,NaN,Y
421,20220508,2,Sun,TEX,AL,26,NYA,AL,27,4,2,54,D,NaN,NaN,NaN,NYC21,40714.0,187,000001300,002000000,33,7,1,0,2,3,0,0,1,1,0,9,0,1,0,0,4,6,2,2,0,0,27,8,0,0,1,0,29,3,0,0,1,2,0,0,1,6,0,6,3,0,1,0,7,3,4,4,1,0,27,12,0,0,0,0,wolcq901,Quinn Wolcott,porta901,Alan Porter,rehaj901,Jeremie Rehak,barrl901,Lance Barrett,NaN,(none),NaN,(none),woodc001,Chris Woodward,boona001,Aaron Boone,richg002,Garrett Richards,kingm002,Michael 

CREATE DATA FRAME CONTAINING THE GAME LOG FOR EVERY GAME OF 'THE MODERN ERA' (1901-PRESENT)

In [12]:
df = pd.DataFrame()
for year in range(1901,2023):
    fname = '/Users/gilliancurtis/Downloads/alldata/gamelogs/gl' +str(year)+'.txt'
    df_temp = pd.read_csv(fname, header=None)
    df_temp.columns = colnames
    df_temp['season'] = year
    df = pd.concat((df, df_temp))

In [13]:
df.shape

(205524, 162)

In [14]:
df.info(max_cols=200)

<class 'pandas.core.frame.DataFrame'>
Index: 205524 entries, 0 to 2429
Data columns (total 162 columns):
 #    Column                Non-Null Count   Dtype  
---   ------                --------------   -----  
 0    date                  205524 non-null  int64  
 1    dblheader_code        205524 non-null  int64  
 2    day_of_week           205524 non-null  object 
 3    team_v                205524 non-null  object 
 4    league_v              205524 non-null  object 
 5    game_no_v             205524 non-null  int64  
 6    team_h                205524 non-null  object 
 7    league_h              205524 non-null  object 
 8    game_no_h             205524 non-null  int64  
 9    runs_v                205524 non-null  int64  
 10   runs_h                205524 non-null  int64  
 11   outs_total            205524 non-null  int64  
 12   day_night             205524 non-null  object 
 13   completion_info       183 non-null     object 
 14   forfeit_info          43 non-null      ob

In [15]:
## Calculate a few useful columns

df['run_diff'] = df['runs_h']-df['runs_v']
df['home_victory'] = (df['run_diff']>0).astype(int)
df['run_total'] = df['runs_h'].copy()+df['runs_v'].copy()
df['date_dblhead'] = (df['date'].astype(str) + df['dblheader_code'].astype(str)).astype(int)

In [16]:
df.home_victory.mean()

0.5387254043323407

Big picture - have outcome, need features

- Would like to start simple, base features on team results over past n games
- Need to do a bit of "data wrangling"
- First, we need to have each team's games in chronological order
- Then can use pandas 'rolling' functionality to aggregate over windows in the past

- Immediate goal, get batting average, on-base percentage, and slugging percentage
- Need "running values" based on the past, not including the current game

In [17]:
# Let's look at a single team's games

df_reds = df.loc[((df.team_v=='CIN') | (df.team_h=='CIN')), :]
df_reds.shape

(19014, 166)

In [18]:
df_reds.sample(50)

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead
1987,19840921,0,Fri,CIN,NL,154,HOU,NL,154,5,2,54,N,NaN,NaN,NaN,HOU02,8676.0,150.0,111001010,000010010,39.0,15.0,3.0,0.0,1.0,5.0,0.0,1.0,0.0,1.0,0.0,2.0,1.0,1.0,0.0,0.0,9.0,2.0,2.0,2.0,0.0,0.0,27.0,11.0,0.0,0.0,0.0,0.0,32.0,6.0,2.0,2.0,0.0,2.0,0.0,0.0,0.0,3.0,0.0,6.0,1.0,0.0,0.0,0.0,6.0,3.0,5.0,5.0,0.0,0.0,27.0,17.0,0.0,0.0,0.0,0.0,mcshj901,John McSherry,brocf901,Fred Brocklander,willc901,Charlie Williams,willb901,Bill Williams,NaN,(none),NaN,(none),rosep001,Pete Rose,lillb101,Bob Lillis,sotom001,Mario Soto,niekj001,Joe Niekro,powet001,Ted Power,cedec001,Cesar Cedeno,sotom001,Mario Soto,niekj001,Joe Niekro,milne001,Eddie Milner,8,oestr001,Ron Oester,4,parkd001,Dave Parker,9,cedec001,Cesar Cedeno,7,peret001,Tony Perez,3,krenw001,Wayne Krenchicki,5,concd001,Dave Concepcion,6,guldb001,Brad Gulden,2,sotom001,Mario Soto,1,dorab001,Bill Doran,4,puhlt001,Terry Puhl,9,garnp001,Phil Garner,5,cruzj001,Jose Cruz,7,davig001,Glenn Davis,3,bassk001,Kevin Bass,8,ashba001,Alan Ashby,2,reync001,Craig Reynolds,6,niekj001,Joe Niekro,1,NaN,Y,1984,-3,0,7,198409210
126,19500508,0,Mon,PHI,NL,21,CIN,NL,17,6,5,54,N,NaN,NaN,NaN,CIN07,5141.0,145.0,020020200,001013000,39.0,12.0,3.0,0.0,1.0,5.0,0.0,0.0,1.0,1.0,0.0,6.0,0.0,0.0,0.0,0.0,8.0,2.0,5.0,5.0,1.0,0.0,27.0,12.0,0.0,0.0,1.0,0.0,32.0,6.0,2.0,0.0,2.0,5.0,0.0,0.0,0.0,5.0,0.0,2.0,0.0,0.0,1.0,0.0,5.0,2.0,5.0,5.0,0.0,0.0,27.0,17.0,2.0,0.0,0.0,0.0,barla901,Al Barlick,robbs901,Scotty Robb,balll901,Lee Ballanfant,donaa901,Augie Donatelli,NaN,(none),NaN,(none),sawye801,Eddie Sawyer,sewel101,Luke Sewell,johnk101,Ken Johnson,wehmh101,Herm Wehmeier,millb103,Bob Miller,NaN,(none),johnk101,Ken Johnson,wehmh101,Herm Wehmeier,ashbr101,Richie Ashburn,8,hamng102,Granny Hamner,6,waite101,Eddie Waitkus,3,ennid101,Del Ennis,9,jonew101,Willie Jones,5,sisld101,Dick Sisler,7,golim101,Mike Goliat,4,silvk101,Ken Silvestri,2,johnk101,Ken Johnson,1,adamb103,Bobby Adams,4,lowrp101,Peanuts Lowrey,7,hattg101,Grady Hatton,5,coopw102,Walker Cooper,2,litwd101,Danny Litwhiler,9,usheb101,Bob Usher,8,klust101,Ted Kluszewski,3,stalv101,Virgil Stallcup,6,wehmh101,Herm Wehmeier,1,NaN,Y,1950,-1,0,11,195005080
1154,19540914,0,Tue,CIN,NL,144,BRO,NL,144,0,4,51,N,NaN,NaN,NaN,NYC15,9877.0,131.0,0.0,20020000x,29.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2

In [19]:
# Write a function to create a team-specific data frame, given the team
def strip_suffix(x, suff):
    if x.endswith(suff):
        return(x[:-len(suff)])
    else:
        return(x)

visit_cols = [col for col in df.columns if not col.endswith('_h')]
visit_cols_stripped = [strip_suffix(col,'_v') for col in visit_cols]
home_cols = [col for col in df.columns if not col.endswith('_v')]
home_cols_stripped = [strip_suffix(col,'_h') for col in home_cols]

## This subsets the game level df by team, to aggregate team statistics easily
## We also create rolling sums with an offset, so that the rollsum number represents
## statistics up to, but not including, the game in question

def create_team_df(team):
    df_team_v = df[(df.team_v==team)]
    opponent = df_team_v['team_h']
    df_team_v = df_team_v[visit_cols]
    df_team_v.columns = visit_cols_stripped
    df_team_v['home_game'] = 0
    df_team_v['opponent'] = opponent

    df_team_h = df[(df.team_h==team)]
    opponent = df_team_h['team_v']
    df_team_h = df_team_h[home_cols]
    df_team_h.columns = home_cols_stripped
    df_team_h['home_game'] = 1
    df_team_h['opponent'] = opponent


    df_team = pd.concat((df_team_h, df_team_v))
    df_team.sort_values(['date', 'game_no'],inplace=True)
    
    for winsize in [162,30]:
        suff = str(winsize)
        for raw_col in ['AB','H','2B','3B','HR','BB','runs','SB','CS','ERR']:
            new_col = 'rollsum_'+raw_col+'_'+suff
            df_team[new_col] = df_team[raw_col].rolling(winsize, closed='left').sum()

        df_team['rollsum_BATAVG_'+suff] = df_team['rollsum_H_'+suff] / df_team['rollsum_AB_'+suff]
        df_team['rollsum_OBP_'+suff] = (df_team['rollsum_H_'+suff] + df_team['rollsum_BB_'+suff]) / (
                                    df_team['rollsum_AB_'+suff]+df_team['rollsum_BB_'+suff])
        df_team['rollsum_SLG_'+suff] = (df_team['rollsum_H_'+suff] + df_team['rollsum_2B_'+suff] 
                                 + 2*df_team['rollsum_3B_'+suff]+ 
                                3*df_team['rollsum_HR_'+suff] ) / (df_team['rollsum_AB_'+suff])
        df_team['rollsum_OBS_'+suff] = df_team['rollsum_OBP_'+suff] + df_team['rollsum_SLG_'+suff]
    
    df_team['season_game'] = df_team['season']*1000 + df_team['game_no']
    df_team.set_index('season_game', inplace=True)
    return(df_team)
df_reds = create_team_df('CIN')
df_reds.sample(10)

,date,dblheader_code,day_of_week,team,league,game_no,runs,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore,AB,H,2B,3B,HR,RBI,SH,SF,HBP,BB,IBB,SO,SB,CS,GIDP,CI,LOB,P_num,ERind,ERteam,WP,balk,PO,ASST,ERR,PB,DP,TP,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id,mgr_name,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id,pitcher_start_name,batter1_name,batter1_id,batter1_pos,batter2_name,batter2_id,batter2_pos,batter3_name,batter3_id,batter3_pos,batter4_name,batter4_id,batter4_pos,batter5_name,batter5_id,batter5_pos,batter6_name,batter6_id,batter6_pos,batter7_name,batter7_id,batter7_pos,batter8_name,batter8_id,batter8_pos,batter9_name,batter9_id,batter9_pos,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,home_game,opponent,rollsum_AB_162,rollsum_H_162,rollsum_2B_162,rollsum_3B_162,rollsum_HR_162,rollsum_BB_162,rollsum_runs_162,rollsum_SB_162,rollsum_CS_162,rollsum_ERR_162,rollsum_BATAVG_162,rollsum_OBP_162,rollsum_SLG_162,rollsum_OBS_162,rollsum_AB_30,rollsum_H_30,rollsum_2B_30,rollsum_3B_30,rollsum_HR_30,rollsum_BB_30,rollsum_runs_30,rollsum_SB_30,rollsum_CS_30,rollsum_ERR_30,rollsum_BATAVG_30,rollsum_OBP_30,rollsum_SLG_30,rollsum_OBS_30
season_game,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1905068,19050704,2,Tue,CIN,NL,68,1,51,D,NaN,NaN,NaN,PIT05,10105.0,120.0,010000000,34.0,8.0,0.0,0.0,0.0,1.0,1.0,-1.0,1.0,1.0,-1.0,3.0,1.0,-1.0,-1.0,0.0,9.0,1.0,-1.0,0.0,1.0,0.0,24.0,15.0,4.0,1.0,0.0,0.0,bausg101,George Bausewine,NaN,(none),NaN,(none),NaN,(none),NaN,(none),NaN,(none),kellj106,Joe Kelley,leevs101,Sam Leever,overo101,Orval Overall,NaN,(none),NaN,(none),overo101,Orval Overall,huggm101,Miller Huggins,4.0,barrs101,Shad Barry,3.0,odwef101,Fred Odwell,7.0,seymc101,Cy Seymour,8.0,corct101,Tommy Corcoran,6.0,sebrj101,Jimmy Sebring,9.0,steih101,Harry Steinfeldt,5.0,schla101,Admiral Schlei,2.0,overo101,Orval Overall,1.0,NaN,D,1905,7,1,9,190507042,0,PIT,5359.0,1347.0,176.0,92.0,17.0,424.0,676.0,171.0,-162.0,311.0,0.251353,0.306242,0.328046,0.634289,978.0,263.0,24.0,16.0,4.0,77.0,142.0,37.0,-30.0,41.0,0.268916,0.322275,0.338446,0.660721
1925015,19250502,0,Sat,CIN,NL,15,3,51,D,NaN,NaN,NaN,PIT06,19000.0,98.0,100000011,33.0,5.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,5.0,4.0,15.0,15.0,0.0,1.0,24.0,5.0,1.0,0.0,0.0,0.0,morac102,Charlie Moran,wilsf901,Frank Wilson,NaN,(none),quige901,Ernie Quigley,NaN,(none),NaN,(none),hendj101,Jack Hendricks,meadl101,Lee Meadows,donop102,Pete Donohue,NaN,(none),barnc102,Clyde Barnhart,donop102,Pete Donohue,dresc101,Chuck Dressen,4,pineb101,Babe Pinelli,5,zitzb101,Billy Zitzmann,7,bresr101,Rube Bressler,3,walkc101,Curt Walker,8,bohns101,Sam Bohne,9,cavei101,Ike Caveney,6,wingi101,Ivey Wingo,2,donop102,Pete Donohue,1,NaN,Y,1925,15,1,21,192505020,0,PIT,5630.0,1640.0,253.0,121.0,39.0,385.0,711.0,113.0,101.0,227.0,0.291297,0.336658,0.400000,0.736658,1055.0,317.0,42.0,29.0,10.0,79.0,158.0,24.0,16.0,44.0,0.300474,0.349206,0.423697,0.772903
1958078,19580713,1,Sun,CIN,NL,78,0,51,D,NaN,NaN,NaN,LOS01,0.0,140.0,0,29.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,10.0,0.0,0.0,1.0,0.0,8.0,2.0,3.0,3.0,1.0,0.0,24.0,11.0,0.0,0.0,1.0,0.0,burkk101,Ken Burkhart,boggd901,Dusty Boggess,sudoe901,Ed Sudol,gormt101,Tom Gorman,NaN,(none),NaN,(none),tebbb101,Birdie Tebbetts,wills102,Stan Williams,newcd101,Don Newcombe,labic101,Clem Labine,furic101,Carl Furillo,newcd101,Don Newcombe,tempj101,Johnny Temple,4,lyncj102,Jerry Lynch,9,robif103,Frank Robinson,7,crowg101,George Crowe,3,hoakd101,Don Hoak,5,baile101,Ed Bailey,2,bellg103,Gus Bell,8,mcmir101,Roy McMillan,6,newcd101,Don Newcombe,1,NaN,Y,1958,3,1,3,195807131,0,LAN,5685.0,1511.0,270.0,39.0,159.0,575.0,748.0,57.0,44.0,121.0,0.265787,0.333227,0.4109

In [20]:
# Make a dictionary that maps a team name to it's data frame
# Create the team level dataframe for each team - put in dict for easy access
team_data_dict = {}
for team in df.team_v.unique():
    team_data_dict[team] = create_team_df(team)

In [21]:
# Go through the rows of the main dataframe, and augment it with home and visiting teams' features
## Create a variety of summarized statistics for each game
## For each game, we look up the home and visiting team in the team
## data dictionary, and then look up the game, and pull the relevant stats

BATAVG_162_h = np.zeros(df.shape[0])
BATAVG_162_v = np.zeros(df.shape[0])
OBP_162_h = np.zeros(df.shape[0])
OBP_162_v = np.zeros(df.shape[0])
SLG_162_h = np.zeros(df.shape[0])
SLG_162_v = np.zeros(df.shape[0])
OBS_162_h = np.zeros(df.shape[0])
OBS_162_v = np.zeros(df.shape[0])
SB_162_h = np.zeros(df.shape[0])
SB_162_v = np.zeros(df.shape[0])
CS_162_h = np.zeros(df.shape[0])
CS_162_v = np.zeros(df.shape[0])
ERR_162_h = np.zeros(df.shape[0])
ERR_162_v = np.zeros(df.shape[0])
BATAVG_30_h = np.zeros(df.shape[0])
BATAVG_30_v = np.zeros(df.shape[0])
OBP_30_h = np.zeros(df.shape[0])
OBP_30_v = np.zeros(df.shape[0])
SLG_30_h = np.zeros(df.shape[0])
SLG_30_v = np.zeros(df.shape[0])
OBS_30_h = np.zeros(df.shape[0])
OBS_30_v = np.zeros(df.shape[0])
SB_30_h = np.zeros(df.shape[0])
SB_30_v = np.zeros(df.shape[0])
CS_30_h = np.zeros(df.shape[0])
CS_30_v = np.zeros(df.shape[0])
ERR_30_h = np.zeros(df.shape[0])
ERR_30_v = np.zeros(df.shape[0])
i=0
for index, row in df.iterrows():
    if i%2500==0:
        print(i)
    home_team = row['team_h']
    visit_team = row['team_v']
    game_index_v = row['season']*1000 + row['game_no_v']
    game_index_h = row['season']*1000 + row['game_no_h']
    BATAVG_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_BATAVG_162']
    BATAVG_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_BATAVG_162']
    OBP_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBP_162']
    OBP_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBP_162']
    SLG_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SLG_162']
    SLG_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SLG_162']
    OBS_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBS_162']
    OBS_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBS_162']
    SB_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SB_162']
    SB_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SB_162']
    CS_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_CS_162']
    CS_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_CS_162']
    ERR_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_ERR_162']
    ERR_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_ERR_162']
    BATAVG_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_BATAVG_30']
    BATAVG_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_BATAVG_30']
    OBP_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBP_30']
    OBP_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBP_30']
    SLG_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SLG_30']
    SLG_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SLG_30']
    OBS_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBS_30']
    OBS_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBS_30']
    SB_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SB_30']
    SB_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SB_30']
    CS_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_CS_30']
    CS_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_CS_30']
    ERR_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_ERR_30']
    ERR_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_ERR_30']
    i+=1

0
2500
5000
7500
10000
12500
15000
17500
20000
22500
25000
27500
30000
32500
35000
37500
40000
42500
45000
47500
50000
52500
55000
57500
60000
62500
65000
67500
70000
72500
75000
77500
80000
82500
85000
87500
90000
92500
95000
97500
100000
102500
105000
107500
110000
112500
115000
117500
120000
122500
125000
127500
130000
132500
135000
137500
140000
142500
145000
147500
150000
152500
155000
157500
160000
162500
165000
167500
170000
172500
175000
177500
180000
182500
185000
187500
190000
192500
195000
197500
200000
202500
205000


In [22]:
## We then put the constructed arrays into the main game level dataframe
df['BATAVG_162_h'] = BATAVG_162_h
df['BATAVG_162_v'] = BATAVG_162_v
df['OBP_162_h'] = OBP_162_h
df['OBP_162_v'] = OBP_162_v
df['SLG_162_h'] = SLG_162_h
df['SLG_162_v'] = SLG_162_v
df['OBS_162_h'] = OBS_162_h
df['OBS_162_v'] = OBS_162_v
df['SB_162_h'] = SB_162_h
df['SB_162_v'] = SB_162_v
df['CS_162_h'] = CS_162_h
df['CS_162_v'] = CS_162_v
df['ERR_162_h'] = ERR_162_h
df['ERR_162_v'] = ERR_162_v
df['BATAVG_30_h'] = BATAVG_30_h
df['BATAVG_30_v'] = BATAVG_30_v
df['OBP_30_h'] = OBP_30_h
df['OBP_30_v'] = OBP_30_v
df['SLG_30_h'] = SLG_30_h
df['SLG_30_v'] = SLG_30_v
df['OBS_30_h'] = OBS_30_h
df['OBS_30_v'] = OBS_30_v
df['SB_30_h'] = SB_30_h
df['SB_30_v'] = SB_30_v
df['CS_30_h'] = CS_30_h
df['CS_30_v'] = CS_30_v
df['ERR_30_h'] = ERR_30_h
df['ERR_30_v'] = ERR_30_v


df.shape

df.sample(25)

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,BATAVG_162_h,BATAVG_162_v,OBP_162_h,OBP_162_v,SLG_162_h,SLG_162_v,OBS_162_h,OBS_162_v,SB_162_h,SB_162_v,CS_162_h,CS_162_v,ERR_162_h,ERR_162_v,BATAVG_30_h,BATAVG_30_v,OBP_30_h,OBP_30_v,SLG_30_h,SLG_30_v,OBS_30_h,OBS_30_v,SB_30_h,SB_30_v,CS_30_h,CS_30_v,ERR_30_h,ERR_30_v
1464,19820812,1,Thu,TOR,AL,114,MIL,AL,113,1,7,51,N,NaN,NaN,NaN,MIL05,0.0,149.0,000100000,02004001x,30.0,5.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,2.0,3.0,7.0,7.0,1.0,0.0,24.0,9.0,0.0,0.0,0.0,0.0,32.0,8.0,2.0,0.0,1.0,7.0,0.0,0.0,0.0,3.0,0.0,7.0,1.0,0.0,0.0,0.0,4.0,1.0,1.0,1.0,0.0,0.0,27.0,7.0,0.0,0.0,0.0,0.0,reilm901,Mike Reilly,barnl901,Larry Barnett,roe-r901,Rocky Roe,merrd901,Durwood Merrill,NaN,(none),NaN,(none),cox-b103,Bobby Cox,kuenh101,Harvey Kuenn,mcclb001,Bob McClure,gottj001,Jim Gott,NaN,(none),thomg001,Gorman Thomas,gottj001,Jim Gott,mcclb001,Bob McClure,garcd001,Damaso Garcia,4,iorgg001,Garth Iorg,5,bonnb001,Barry Bonnell,8,barfj001,Jesse Barfield,9,upshw001,Willie Upshaw,3,martb001,Buck Martinez,2,robel001,Leon Roberts,10,johnt101,Tony Johnson,7,grifa001,Alfredo Griffin,6,molip001,Paul Molitor,5,younr001,Robin Yount,6,coopc001,Cecil Cooper,3,simmt001,Ted Simmons,2,oglib001,Ben Oglivie,7,thomg001,Gorman Thomas,8,hower001,Roy Howell,10,edwam102,Marshall Edwards,9,gantj001,Jim Gantner,4,NaN,Y,1982,6,1,8,198208121,0.271495,0.259698,0.327404,0.311250,0.436675,0.379166,0.764079,0.690416,69.0,98.0,57.0,88.0,127.0,150.0,0.280534,0.282588,0.333333,0.321942,0.449427,0.421503,0.782761,0.743446,13.0,30.0,7.0,17.0,17.0,15.0
1347,20210711,0,Sun,OAK,AL,92,TEX,AL,90,4,1,54,D,NaN,NaN,NaN,ARL03,30531.0,159.0,020101000,000010000,33.0,6.0,1.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,1.0,0.0,0.0,2.0,3.0,1.0,1.0,0.0,0.0,27.0,10.0,0.0,0.0,1.0,0.0,31.0,5.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,4.0,1.0,0.0,1.0,0.0,6.0,4.0,4.0,4.0,0.0,0.0,27.0,16.0,1.0,0.0,0.0,0.0,mosce901,Edwin Moscoso,drecb901,Bruce Dreckman,lentn901,Nic Lentz,westj901,Joe West,NaN,(none),NaN,(none),melvb001,Bob Melvin,woodc001,Chris Woodward,bassc001,Chris Bassitt,allak001,Kolby Allard,trivl001,Lou Trivino,lowrj001,Jed Lowrie,bassc001,Chris Bassitt,allak001,Kolby Allard,laurr001,Ramon Laureano,8,andre001,Elvis Andrus,6,olsom001,Matt Olson,3,chapm001,Matt Chapman,5,lowrj001,Jed Lowrie,4,murps001,Se

In [23]:
df.to_csv('df_bp1.csv', index=False)